# Параллельное обучение бустинга на разных параметрах

In [212]:
%%time
# СОЗДАДИМ ТАБЛИЧКУ ВСЕХ ПАРАМЕТРОВ ДЛЯ ПЕРЕБОРА

all_parameters = [(x0,x1,x2,x3,x4,x5)
                  for x0 in range(500, 1001, 50) 
                  for x1 in range(3, 5, 1)
                  for x2 in [x/1000 for x in range(3, 20, 5)]
                  for x3 in [0.7,0.8] 
                  for x4 in [80,100,120] 
                  for x5 in [0.8,1,1.2]
                  ]

print(len(all_parameters))

table_with_parameters = pd.DataFrame(all_parameters, columns=[
    'n_estimators','max_depth','learning_rate','subsample','min_child_weight','scale_pos_weight'
]).astype( dtype={
                 'n_estimators' : int, 
                 'max_depth': int,
                 'learning_rate': float,
                 'subsample': float,
                 'min_child_weight': int,
                 'scale_pos_weight': float})

1584
CPU times: user 6.01 ms, sys: 119 µs, total: 6.13 ms
Wall time: 4.73 ms


In [213]:
# СОЗДАЕМ ПОЛЕ СО ВСЕМИ ПАРАМЕТРАМИ В ВИДЕ СПИСКА

table_with_parameters['all_param_list'] = table_with_parameters[
    ['n_estimators', 'max_depth','learning_rate','subsample','min_child_weight','scale_pos_weight']
    ].apply(lambda x : list([x[0],x[1],x[2],x[3],x[4],x[5]]), axis=1)

In [214]:
# ФУНКЦИЯ ОБУЧЕНИЯ БУСТИНГА ДЛЯ ЗАДАННЫХ ПАРАМЕТРОВ -- ИМЕННО ЕЁ БУДЕТ ВЫЧИСЛЯТЬ ПАРАЛЛЕЛЬНО
# ВЫДАЕТ СПИСОК ИЗ ДЖИНИ НА КАЖДОМ ИЗ СЕГМЕНТОВ -- ТРЕЙН, ТЕСТ, ООТ

def learning_boost(x0,x1,x2,x3,x4,x5):
    clf = xgb.XGBClassifier( 
        objective='binary:logistic',
        n_estimators=x0,
        max_depth=x1,
        learning_rate=x2, 
        subsample=x3,
        colsample_bytree=0.8,
        eval_metric='auc',
        tree_method='hist',
        reg_lambda=1,
        min_child_weight=x4,
        n_jobs=-1,
        importance_type='weight',
        random_state=123,
        scale_pos_weight=x5,
        max_delta_step =0
        )
    h = clf.fit(X_train, y_train)
    gini_tr = roc_auc_score(y_train,clf.predict_proba(X_train)[:,1])*2-1
    gini_te = roc_auc_score(y_test,clf.predict_proba(X_test)[:,1])*2-1
    gini_oot = roc_auc_score(y_oot,clf.predict_proba(X_oot)[:,1])*2-1
    
    return [gini_tr,gini_te,gini_oot]

# Создание таблицы результатов буста

In [215]:
print(len(variables), 'variables') # ПЕРЕМЕННЫЕ

X_train=train[variables]
X_test=test[variables]
X_oot=oot[variables]

y_train=train['is_target']
y_test=test['is_target']
y_oot=oot['is_target']

198 variables


In [216]:
# ИСПОЛЬЗУЕМ НАДСТРОЙКУ ДЛЯ БЫСТРОГО ПАРАЛЛЕЛЬНОГО ВЫЧИСЛЕНИЯ
import ray
import modin.pandas as pd

In [217]:
%%time
# ПАРАЛЛЕЛЬНОЕ ВЫЧИСЛЕНИЕ

table_with_parameters['all_gini_result'] = table_with_parameters[
    ['n_estimators', 'max_depth','learning_rate','subsample','min_child_weight','scale_pos_weight']
    ].apply(lambda x : learning_boost(int(x[0]),int(x[1]),float(x[2]),float(x[3]),int(x[4]),float(x[5])), axis=1)

CPU times: user 3d 11h 41min 1s, sys: 1h 40min 28s, total: 3d 13h 21min 30s
Wall time: 1h 47min 21s


In [218]:
# РАСПАКОВКА РЕЗУЛЬТАТОВ

table_with_parameters['gini_tr'] = table_with_parameters['all_gini_result'].apply(lambda x : x[0])
table_with_parameters['gini_te'] = table_with_parameters['all_gini_result'].apply(lambda x : x[1])
table_with_parameters['gini_oot'] = table_with_parameters['all_gini_result'].apply(lambda x : x[2])

table_with_parameters = table_with_parameters.drop(['all_param_list','all_gini_result'], axis = 1)

In [219]:
table_with_parameters['diff_tr_te'] = abs(table_with_parameters.gini_tr - table_with_parameters.gini_te)
table_with_parameters['diff_tr_oot'] = abs(table_with_parameters.gini_tr - table_with_parameters.gini_oot)

# Итог

In [ ]:
# Получаем таблицу результатов бустинга на разных параметрах, при этом работа давольно быстрая 
# (может занимать 1 час на более чем 3000 варинтов параметров)

In [220]:
table_with_parameters.sort_values(by='gini_tr',ascending=False).reset_index()

,index,n_estimators,max_depth,learning_rate,subsample,min_child_weight,scale_pos_weight,gini_tr,gini_te,gini_oot,diff_tr_te,diff_tr_oot
0,1577,1000,4,0.018,0.8,80,1.2,0.567392,0.289104,0.292280,0.278289,0.275112
1,1568,1000,4,0.018,0.7,80,1.2,0.560578,0.289685,0.288748,0.270893,0.271830
2,1433,950,4,0.018,0.8,80,1.2,0.559097,0.291125,0.292402,0.267973,0.266695
3,1424,950,4,0.018,0.7,80,1.2,0.553410,0.290907,0.288852,0.262503,0.264558
4,1580,1000,4,0.018,0.8,100,1.2,0.552910,0.288811,0.292980,0.264098,0.259930
...,...,...,...,...,...,...,...,...,...,...,...,...
1579,9,500,3,0.003,0.8,80,0.8,0.338139,0.251466,0.273801,0.086673,0.064338
1580,6,500,3,0.003,0.7,120,0.8,0.337835,0.257428,0.273092,0.080407,0.064743
1581,159,550,3,0.003,0.8,120,0.8,0.336698,0.253760,0.272807,0.082938,0.063892
1582,12,500,3,0.003,0.8,100,0.8,0.336179,0.253429,0.273368,0.082750,0.062811
